In [1456]:
import pandas as pd
import numpy as np


In [1457]:
root_df=pd.read_csv('train.csv')
root_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [1458]:
root_df.drop('Id',axis=1,inplace=True)
root_df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Handle the Null

In [1459]:
null_cols = root_df.columns[root_df.isnull().any()]
root_df[null_cols].isnull().sum()

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

LotFrontage;backfill by LotArea

Alley;Drop

MasVnrType;backfill

MasVnrArea;backfill

BsmtQual;nan is no basement label

BsmtCond;nan is no basement label

BsmtExposure;nan is no basement label

BsmtFinType1;nan is no basement label

BsmtFinType2;nan is no basement label

Electrical;backfill

FireplaceQu;nan is no fireplace

GarageType

GarageYrBlt

GarageFinish

GarageQual

GarageCond

PoolQC;Drop

Fence;Drop

MiscFeature;Drop


In [1460]:
df=root_df.drop(['Alley','PoolQC','Fence','MiscFeature'],axis=1)

# df['remodeling']=root_df['YearBuilt']==root_df['YearRemodAdd']

df.sort_values(by=['LotArea'],inplace=True)
df['LotFrontage']=df['LotFrontage'].fillna(method='backfill')
df.sort_index(inplace=True)
# df[['MasVnrType','MasVnrArea']].loc[df['MasVnrType']=='None'].count()

df['MasVnrType'].fillna(method='backfill',inplace=True)
df['MasVnrArea'].fillna(method='backfill',inplace=True)

df[['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']]=df[['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']].fillna('NA')

df['Electrical'].fillna(method='backfill',inplace=True)

df['FireplaceQu'].fillna('NA',inplace=True)

df[['GarageType','GarageFinish','GarageQual','GarageCond']]=df[['GarageType','GarageFinish','GarageQual','GarageCond']].fillna('NA')

df['GarageYrBlt'].fillna(0,inplace=True)

df.columns[df.isnull().any()]

Index([], dtype='object')

### Split the data into features and label

In [1461]:
object_cols = df.select_dtypes(include=['object']).columns
num=len(object_cols)
cols = list(df.columns[1:])
for c in object_cols:
    # move the selected column to the front
    cols.insert(0, cols.pop(cols.index(c)))
df = df[cols]

features_onehot=df.iloc[:,:num].values
features_num=df.iloc[:,num:-1].values
labels=df.iloc[:,-1].values
print(features_num.shape,features_onehot.shape)


(1460, 35) (1460, 39)


In [1462]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder,StandardScaler

onehotencoder = OneHotEncoder(drop='first')
features_onehot= onehotencoder.fit_transform(features_onehot).toarray()
print(features_num.shape,features_onehot.shape)
scaler=StandardScaler()
features_num= scaler.fit_transform(features_num)
# print(features_num)
# print(features_onehot)
features=np.concatenate((features_onehot,features_num),axis=1)



(1460, 35) (1460, 210)


In [1463]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size = 0.2, random_state = 0)

In [1464]:
from sklearn.linear_model import *
best_sc=0
best_alpha=0
for i in np.arange(0,10,0.1):
    regressor = SGDRegressor(alpha=i,penalty='l1',random_state=0)
    regressor = regressor.fit(X_train, Y_train)
    y_pred = regressor.predict(X_test)
    sc=regressor.score(X_test , Y_test)
    if sc>best_sc:
        best_sc=sc
        best_alpha=i
print(best_alpha)
print(best_sc)

7.1000000000000005
0.7107760312182742


In [1465]:
regressor = SGDRegressor(alpha=best_alpha,penalty='l1',random_state=0)
regressor = regressor.fit(X_train, Y_train)
y_pred = regressor.predict(X_test)
print(regressor.score(X_train , Y_train))
print(regressor.score(X_test , Y_test))


0.9072651605121285
0.7107760312182742


### Apply the model

In [1466]:
root_df=pd.read_csv('test.csv')

df=root_df.drop(['Alley','PoolQC','Fence','MiscFeature'],axis=1)

# df['remodeling']=root_df['YearBuilt']==root_df['YearRemodAdd']

df.sort_values(by=['LotArea'],inplace=True)
df['LotFrontage']=df['LotFrontage'].fillna(method='backfill')
df.sort_index(inplace=True)
# df[['MasVnrType','MasVnrArea']].loc[df['MasVnrType']=='None'].count()

df['MasVnrType'].fillna(method='backfill',inplace=True)
df['MasVnrArea'].fillna(method='backfill',inplace=True)

df[['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']]=df[['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']].fillna('NA')

df['Electrical'].fillna(method='backfill',inplace=True)

df['FireplaceQu'].fillna('NA',inplace=True)

df[['GarageType','GarageFinish','GarageQual','GarageCond']]=df[['GarageType','GarageFinish','GarageQual','GarageCond']].fillna('NA')

df['GarageYrBlt'].fillna(0,inplace=True)

df.drop('Id',axis=1,inplace=True)
df.fillna(method='backfill',inplace=True)

object_cols = df.select_dtypes(include=['object']).columns
num=len(object_cols)
cols = list(df.columns[1:])
for c in object_cols:
    # move the selected column to the front
    cols.insert(0, cols.pop(cols.index(c)))
df = df[cols]

features_onehot=df.iloc[:,:num].values
features_num=df.iloc[:,num:].values
print(features_num.shape,features_onehot.shape)


(1459, 35) (1459, 39)


In [1467]:
features_onehot= onehotencoder.transform(features_onehot).toarray()
features_num= scaler.transform(features_num)
features=np.concatenate((features_onehot,features_num),axis=1)
print(features_num.shape,features_onehot.shape)


(1459, 35) (1459, 210)


In [1468]:
y_pred = regressor.predict(features)
root_df['SalePrice']=y_pred
with open('expect.csv','w') as f:
    root_df[['Id','SalePrice']].to_csv(f,index=False)
root_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,108109.977510
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,162359.162914
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,180105.429268
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,191152.626501
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,191874.475384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,83740.690530
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,71009.214956
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,176632.279469
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,112870.726038
